https://www.kaggle.com/c/data-science-for-good-city-of-los-angeles

Data Science for Good: City of Los Angeles

Help the City of Los Angeles to structure and analyze its job descriptions

The City of Los Angeles faces a big hiring challenge: 1/3 of its 50,000 workers are eligible to retire by July of 2020. The city has partnered with Kaggle to create a competition to improve the job bulletins that will fill all those open positions.
Problem Statement

The content, tone, and format of job bulletins can influence the quality of the applicant pool. Overly-specific job requirements may discourage diversity. The Los Angeles Mayor’s Office wants to reimagine the city’s job bulletins by using text analysis to identify needed improvements.

The goal is to convert a folder full of plain-text job postings into a single structured CSV file and then to use this data to: (1) identify language that can negatively bias the pool of applicants; (2) improve the diversity and quality of the applicant pool; and/or (3) make it easier to determine which promotions are available to employees in each job class.

In [1]:


import numpy as np 
import pandas as pd 


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



In [2]:
import glob
def job_to_df(
path='../input/data-science-for-good-city-of-los-angeles/cityofla/CityofLA/Job Bulletins/*.txt',
col_name='raw_job_text'):
    list_job = []
    
    files = glob.glob(path)
    for file in files:
      
        with open(file,'r',errors='replace') as f:
            content=f.read()
            list_job.append(content)
    else:
        return pd.DataFrame({col_name:list_job})
job_to_df()

In [3]:
def cleaned_text(text):
    return text.replace("\n","").replace("\t","").strip()
text='''
line1
line2   line2.1
'  test '
'''
cleaned_text(text)

In [4]:
def preprocessing(dataframe):

    dataframe['raw_job_text'] = dataframe['raw_job_text'].apply(
        lambda x: x.lstrip())
    return dataframe
preprocessing(job_to_df())

In [5]:

def job_title(dataframe):
    
    dataframe['JOB_CLASS_TITLE'] = dataframe['raw_job_text'].apply(
        lambda x: x.split('\n', 1)[0])
    dataframe['JOB_CLASS_TITLE'] = dataframe['JOB_CLASS_TITLE'].apply(
        lambda x: cleaned_text(x))
    return dataframe
job_title(job_to_df())

In [7]:
import re
def job_class_code(text):
    """
    This class extract job class code
    """
    match = re.search('Class Code: (\d+)', text)
    class_code = None
    try:
        class_code = match.group(1)
    except:
        class_code = None
    return class_code
job_class_code(text='Class Code: 1234')

In [8]:
def class_code(dataframe):
    
    temp = dataframe['raw_job_text'].apply(lambda x: ' '.join(x.split()))
   
    dataframe['JOB_CLASS_NO'] = temp.apply(lambda x: job_class_code(x))
    return dataframe
class_code(job_to_df())

In [9]:
def exam_type(text):
    
    """
    Extract entire exam type section
    """
    
    et = ""

    result= re.search(
        "(Class Code:|Class  Code:)(.*)(ANNUAL SALARY|ANNUALSALARY)",
        text)
    
    shortContent=''
    if result:
        shortContent=result.group(2).strip()
        result= re.search(
            "\(+(.*?)\)", shortContent,flags=re.IGNORECASE)
        if result:
            
            et=result.group(1).strip()
    return et

In [10]:
def exam_type_extraction(dataframe):
    
    temp = dataframe['raw_job_text'].apply(lambda x: ' '.join(x.split()))
    
    dataframe['TEMP_EXAM_TYPE'] = temp.apply(lambda x: exam_type(x))
    return dataframe
exam_type_extraction(job_to_df())

In [11]:
def salary(text):
   
    ss = ''
    salary_notes = ''
    result=re.search(
        "(ANNUAL SALARY|ANNUALSALARY)(.*?)DUTIES", text)
    if result:
        salContent= result.group(2).strip()
        if "NOTE:" in salContent or "NOTES:" in salContent:
           
            result=re.search(
                "(.*?)(NOTE:|NOTES:)",
                salContent,flags=re.IGNORECASE)
            if result:
                s=result.group(1).strip()  
           
            result= re.search(
                "(NOTE:|NOTES:)(.*)",
                salContent,flags=re.IGNORECASE)
            if result:
                salary_notes= result.group(2).strip()
        else:
            s = salContent
    else:
        
        result=re.search(
            "(ANNUAL SALARY|ANNUALSALARY)(.*?)REQUIREMENT",
            text,flags=re.IGNORECASE)
        if result:
            salContent= result.group(2).strip()
            if "NOTE:" in salContent or "NOTES:" in salContent:
                
                result=re.search(
                    "(.*?)(NOTE:|NOTES:)",
                    salContent,flags=re.IGNORECASE)
                if result:
                    s=result.group(1).strip()
                
                result= re.search(
                    "(NOTE:|NOTES:)(.*)",
                    salContent,flags=re.IGNORECASE)
                if result:
                    salary_notes= result.group(2).strip()
            else:
                s= salContent
    salary_text = "|||||||||||||||".join([s, salary_notes])
    return salary_text

In [12]:
def salary_extraction(dataframe):
    
    temp = dataframe['raw_job_text'].apply(lambda x: ' '.join(x.split()))
    
    dataframe['TEMP_SALARY'] = temp.apply(lambda x: salary(x))
    return dataframe
salary_extractionsalary_extraction(job_to_df())

In [13]:
def duties(text):
    """
    Extract job duties section
    """
    '''
    DUTIES A Pipefitter does skilled work in the installation, replacement, maintenance, and repair of industrial steam, oil,
    air, gas, water, steam heating and other pipe systems, including high pressure and high temperature systems. 
    REQUIREMENT
    '''
    d=''
    result=duties= re.search("DUTIES(.*?)REQUIREMENT", text)
    if result:
        duties= result.group(1).strip()
    return d

In [14]:
def duty_extraction(dataframe):
    # remove all extra white spaces
    temp = dataframe['raw_job_text'].apply(lambda x: ' '.join(x.split()))
    
    dataframe['JOB_DUTIES'] = temp.apply(lambda x: duties(x))
    return dataframe
duty_extraction(job_to_df())

In [15]:
def requirements(text):
   
    req='|'.join(["REQUIREMENT/MIMINUMUM QUALIFICATION",
                  "REQUIREMENT/MINUMUM QUALIFICATION",
                  "REQUIREMENT/MINIMUM QUALIFICATION",
                  "REQUIREMENT/MINIMUM QUALIFICATIONS",
                  "REQUIREMENT/ MINIMUM QUALIFICATION",
                  "REQUIREMENTS/MINUMUM QUALIFICATIONS",
                  "REQUIREMENTS/ MINIMUM QUALIFICATIONS",
                  "REQUIREMENTS/MINIMUM QUALIFICATIONS",
                  "REQUIREMENTS/MINIMUM REQUIREMENTS",
                  "REQUIREMENTS/MINIMUM QUALIFCATIONS",
                  "MINIMUM REQUIREMENTS:",
                  "REQUIREMENTS",
                  "REQUIREMENT"])
    
    result= re.search(f"({req})(.*)(WHERE TO APPLY|HOW TO APPLY)", text)
    requirements=''
    if result:
        requirements = result.group(2).strip()
    return requirements

In [16]:
def requirement_extraction(dataframe):
    # remove all extra white spaces
    temp = dataframe['raw_job_text'].apply(lambda x: ' '.join(x.split()))
    
    dataframe['TEMP_REQUIREMENTS'] = temp.apply(lambda x: requirements(x))
    return dataframe
requirement_extraction(job_to_df())

In [17]:
def location(text):
    
    """
    Extract entire 'WHERE TO APPLY' section
    """
    '''
    WHERE TO APPLY Applications will only be accepted on-line. When you are viewing the on-line job bulletin of your choice, 
    simply scroll to the top of the page and select the "Apply" icon. On-line job bulletins are also available 
    at https://www.governmentjobs.com/careers/lacity. NOTE: Applicants are urged to apply early to ensure you have time to 
    resolve any technical issues you may encounter. APPLICATION DEADLINE
    '''
    
    where_to_apply = ''
    result= re.search(
        "(HOW TO APPLY|WHERE TO APPLY)(.*)(APPLICATION DEADLINE|APPLICATION PROCESS)",
        text)
    if result:
        where_to_apply= result.group(2).strip()
    else:
        '''
        WHERE TO APPLY THE SELECTION PROCESS IS STARTED by faxing a letter stating your desire to apply for Police Specialist. The 
        ...to schedule the necessary steps. SELECTION PROCESS
        '''
        result= re.search(
            "(HOW TO APPLY|WHERE TO APPLY)(.*)(SELECTION PROCESS|SELELCTION PROCESS)",
            text)
        if result:
            where_to_apply= result.group(2).strip()
    return where_to_apply

In [18]:
def location_extraction(dataframe):
    # remove all extra white spaces
    temp = dataframe['raw_job_text'].apply(lambda x: ' '.join(x.split()))
    
    dataframe['WHERE_TO_APPLY'] = temp.apply(lambda x: _where_to_apply(x))
    return dataframe
location_extraction(job_to_df())

In [19]:
def deadline(text):
    
    d=''
    result= re.search(
        "(APPLICATION DEADLINE|APPLICATION PROCESS)(.*?)(SELECTION PROCESS|SELELCTION PROCESS)",
        text)
    if result:
        d= result.group(2).strip()
    else:
        
        result= re.search(
            "(APPLICATION DEADLINE|APPLICATION PROCESS)(.*?)(Examination Weight:)",
            text)
        if result:
            d= result.group(2).strip()
            
    return d

In [20]:
def deadline_extraction(dataframe):
    # remove all extra white spaces
    temp = dataframe['raw_job_text'].apply(lambda x: ' '.join(x.split()))
    
    dataframe['DEADLINE'] = temp.apply(lambda x: deadline(x))
    return dataframe
deadline_extraction(job_to_df())

In [21]:
def selection_process(text):
    
    """
    Extract selectioin process section
    """
    '''
    SELECTION PROCESS Examination Weight: Multiple-Choice Test ...
    time, and location of their interview (anticipated to be held in late March 2017). APPOINTMENT
    '''
    sp=''
    result=sp= re.search(
        "(SELECTION PROCESS|Examination Weight:)(.*)(APPOINTMENT|APPOINTMENT IS SUBJECT TO:)",
        text)
    if result:
        sp= result.group(2).strip()
    else:
        '''
        SELECTION PROCESS Examination Weight: Multiple-Choice Test ...
        '''
        result=sp= re.search(
            "(SELECTION PROCESS|Examination Weight:)(.*)",
            text)
        if result:
            sp= result.group(2).strip()
            
    return sp

In [22]:
def extract_selection_process(dataframe):
    # remove all extra white spaces
    temp = dataframe['raw_job_text'].apply(lambda x: ' '.join(x.split()))
    
    dataframe['SELECTION_PROCESS'] = temp.apply(lambda x: selection_process(x))
    return dataframe
extract_selection_process(job_to_df())

In [23]:
def OpenDate(text):
    
    """
    Extract entire job open date section
    """
    '''
    Class Code:       1590
    Open Date:  06-01-18(Exam Open to Current City Employees)
    ANNUAL SALARY
    '''
    open_date = ''
    result= re.search(
        "(Class Code:|Class  Code:)(.*)(ANNUAL SALARY|ANNUALSALARY)",
        text)
    
    shortContent=''
    if result:
        shortContent=result.group(2).strip()
        #Open Date:  06-01-18 REVISED
        result= re.search(
            "Open Date:(.*)REVISED",
            shortContent,flags=re.IGNORECASE)
        if result:
            open_date=result.group(1).strip()
        if open_date=='':
            #Open Date:  06-01-18(Exam Open to Current City Employees)
            result= re.search(
                "Open Date:(.*)\(Exam",
                shortContent,flags=re.IGNORECASE)
            if result:
                open_date=result.group(1).strip()
        if open_date=='':
            #Open Date:  06-01-18
            result= re.search(
                "Open Date:(.*)",
                shortContent,flags=re.IGNORECASE)
            if result:
                open_date=result.group(1).strip()
    return open_date

In [24]:
def OpenDate_extraction(dataframe):
    # remove all extra white spaces
    temp = dataframe['raw_job_text'].apply(lambda x: ' '.join(x.split()))
    
    dataframe['OPEN_DATE'] = temp.apply(lambda x: OpenDate(x))
    return dataframe
OpenDate_extraction(job_to_df())

In [25]:
# first let's convert folder of raw text job bulletins
# to pandas dataframe
data = job_to_df()

# do some initial text cleaning
data = preprocessing(data)

data = job_title(data) 

data = class_code(data) 

data = OpenDate_extraction(data) 

data = exam_type_extraction(data) 

data = salary_extraction(data) 

data = duty_extraction(data) 

data = requirement_extraction(data) 

data = location_extraction(data) 

data = deadline_extraction(data) 

data = extract_selection_process(data) 

In [26]:

data['raw_clean_job_text'] = data['raw_job_text'].apply(
    lambda x: cleaned_text(x))

In [27]:

data.head(2)

In [28]:
data.tail(2)

In [29]:
data.shape

In [30]:
def base_info(data):
  print(f' Zero \n:{data.isnull().sum()}\n')
  print(f'NaN :\n {data.isna().sum()}\n')
  print(f"type of data:\n {data.dtypes}\n")
base_info(data)

In [31]:
data.info(verbose=True)

In [32]:
data.info(verbose=False)

In [33]:
data['JOB_CLASS_NO'].describe()

In [34]:
print(data['JOB_CLASS_NO'].isnull().sum())
print(data['JOB_CLASS_NO'].isna().sum())

In [35]:
data['JOB_CLASS_NO'].fillna(0, inplace=True)

In [36]:
data['JOB_CLASS_NO'] = data['JOB_CLASS_NO'].astype(str).astype(int)

In [37]:
data['JOB_CLASS_NO'].describe()

In [38]:
mas_pronouns =["he", "his", "him", "himself"]
fem_pronouns =["she", "her", "herself"]

In [39]:
for i in mas_pronouns:
    sum= data.TEMP_REQUIREMENTS.str.contains(r'i').sum() + data.SELECTION_PROCESS.str.contains(r'i').sum()
    print(f'{i} : {sum}')

In [40]:
for i in fem_pronouns:
    sum= data.TEMP_REQUIREMENTS.str.contains(r'i').sum() + data.SELECTION_PROCESS.str.contains(r'i').sum()
    print(f'{i} : {sum}')